In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [6]:
log_dir=Path(r'./final_results')
#log_dir=Path(r'./final_results_rerun')
results_filename = 'all_model_metrics.json'
runs=[]
for run_instance in log_dir.iterdir():
    if run_instance.is_dir():
        iteration_result = Path(run_instance / results_filename)
        # iteration_result = Path(run_instance / '30d_mortality' / results_filename)
        if iteration_result.exists():
            result = json.load(open(iteration_result))
            runs.append(result)

In [7]:
{model: np.mean(results[0]) for model, results in runs[0]['roc_auc'].items() if model != "DummyClassifier"}

{'DecisionTreeClassifier': 0.660625,
 'RandomForestClassifier': 0.725625,
 'GradientBoostingClassifier': 0.6525,
 'LogisticRegression': 0.655,
 'SVC': 0.665,
 'LinearSVC': 0.6325000000000001,
 'MLPClassifier': 0.6575}

In [8]:
aggregated_cv_results = {}
for metric in ['roc_auc', 'prc_auc', 'mcc']:
    single_metric_results = [single_result[metric] for single_result in runs]
    mean_cv_results = [{model: np.mean(results[0]) for model, results in single_seed_results.items() if model != "DummyClassifier"} for single_seed_results in single_metric_results]
    rearrange_for_model = {model: [single_cv_result[model] for single_cv_result in mean_cv_results] for model in mean_cv_results[0].keys()}
    aggregated_per_model = {model: (np.mean(results), np.min(results), np.max(results)) for model, results in rearrange_for_model.items()}
    print(metric)
    print(aggregated_per_model)
    print()
    aggregated_cv_results[metric] = aggregated_per_model

roc_auc
{'DecisionTreeClassifier': (0.6147875, 0.5, 0.7725), 'RandomForestClassifier': (0.6680375000000001, 0.48625, 0.860625), 'GradientBoostingClassifier': (0.7226249999999999, 0.575, 0.8625), 'LogisticRegression': (0.7447625000000001, 0.6287499999999999, 0.86125), 'SVC': (0.7313124999999999, 0.4275, 0.845), 'LinearSVC': (0.7074375, 0.58375, 0.81), 'MLPClassifier': (0.7153625000000001, 0.45625, 0.845)}

prc_auc
{'DecisionTreeClassifier': (0.26914884597916877, 0.07736531614240591, 0.47730486425339363), 'RandomForestClassifier': (0.21556820453802683, 0.04229961215255333, 0.41025012569130215), 'GradientBoostingClassifier': (0.22600159044150822, 0.1258871336996337, 0.4191368003771255), 'LogisticRegression': (0.22985537990578186, 0.10080820221445222, 0.39637786685820575), 'SVC': (0.22701524621280642, 0.05095744501027009, 0.34177884889233573), 'LinearSVC': (0.23222890073018848, 0.08898476660401002, 0.39259271859891054), 'MLPClassifier': (0.28373075918179064, 0.12394502516453736, 0.41465086

In [9]:
{metric: (np.min([entry[0] for entry in model_results.values()]), np.max([entry[0] for entry in model_results.values()])) for metric, model_results in aggregated_cv_results.items()}

{'roc_auc': (0.6147875, 0.7447625000000001),
 'prc_auc': (0.21556820453802683, 0.28373075918179064),
 'mcc': (0.0011569640361619405, 0.129522320212816)}